In [1]:
import pandas as pd
import xlsxwriter
from random import randint
from time import sleep
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
# start with 2008 - when BLK is available
seasons = [year for year in range(2008, 2025) if year != 2005]
skater_stats = ['Player', 'Age', 'Pos', 'GP', 'G', 'A', '+/-', 'PIM', 'PPP', 'ATOI', 'S', 'BLK', 'HIT']
goalie_stats = ['W', 'GAA', 'SV%']

In [3]:
# GOAL: append each season's dataframe to the original, parent dataframe

# Initialize an empty DataFrame
stats_df = pd.DataFrame()

for season in seasons:

    # create season stats URL
    stats_url = f"https://www.hockey-reference.com/leagues/NHL_{season}_skaters.html"

    # Read data from website
    stats_dfs = pd.read_html(stats_url)
    temp_df = stats_dfs[0]

    # Initialize temporary DataFrame
    list_of_columns = [col[1] for col in temp_df.columns]
    temp_df.columns = list_of_columns
    first_column = temp_df.columns[0]
    temp_df.set_index(first_column, inplace=True)

    # Rename columns as necessary
    new_column_names = {11: 'G_EV', 12: 'G_PP', 13: 'G_SH', 14:
        'GWG', 15: 'A_EV', 16: 'A_PP', 17: 'A_SH'}
    for col in new_column_names:
        temp_df.columns.values[col] = new_column_names[col]

    # Convert columns to numeric as necessary
    temp_df.drop(columns=['FOW', 'FOL', 'FO%'], errors='ignore', inplace=True)
    indices_to_numeric = [col for col in range(4, 24) if col != 21]
    cols_to_numeric = [temp_df.columns[col] for col in indices_to_numeric]
    for column in cols_to_numeric:
        temp_df[column] = pd.to_numeric(temp_df[column], errors='coerce')

    # Create PPP statistic by adding PP goals to PP assists 
    temp_df['PPP'] = temp_df['G_PP'] + temp_df['A_PP']

    # Filter out unneeded columns
    filtered_columns = [col for col in skater_stats if col in temp_df.columns]
    temp_df = temp_df[filtered_columns].copy()  # create a copy to avoid warning message

    # Add season column
    temp_df.insert(0, 'Season', f'{str(season - 1)}-{str(season)[2:]}')

    # Remove duplicate player ID's
    duplicates = temp_df.index.duplicated(keep='first')
    temp_df = temp_df.loc[~duplicates]

    # Get rid of repeated column headers
    temp_df.dropna(axis=0, how='any', inplace=True)

    # Concatenate temp_df to the end of stats_df
    stats_df = pd.concat([stats_df, temp_df], ignore_index=True)

    # delay next iteration 
    sleep(randint(10, 25))

# Export dataframe to excel for future use
stats_df.to_excel(r"C:\Users\naray\Documents\Coding Projects\stats_df.xlsx", engine='xlsxwriter')


In [12]:
# Dataframe filtering

# Store dataframe from excel
stats_df = pd.read_excel(r"C:\Users\naray\Documents\Coding Projects\stats_df.xlsx")

stats_df.drop(columns=[stats_df.columns[0]], inplace=True)

# Create a new column combining 'Name' and 'Position'
stats_df['Name_Pos'] = stats_df['Player'] + "_" + stats_df['Pos']

# Get unique combinations of 'Name_Pos'
unique_name_pos = stats_df['Name_Pos'].unique()

# Generate a dictionary with random numbers as IDs for each 'Name_Pos' combination
name_pos_id_map = {combo: randint(10000, 99999) for combo in unique_name_pos}

# Apply the mapping to assign each player-team combination a random ID
stats_df['ID'] = stats_df['Name_Pos'].map(name_pos_id_map)
id_col = stats_df.pop('ID')
stats_df.insert(0, 'ID', id_col)

stats_df.drop(columns=['Name_Pos'], inplace=True)
# Remove GP outliers 
min_gp = 10
stats_df = stats_df[stats_df['GP'] >= min_gp]


# convert ATOI to seconds 
def atoi_to_seconds(col):
    minutes, seconds = map(int, col.split(':'))
    return minutes * 60 + seconds


stats_df['ATOI'] = stats_df['ATOI'].apply(atoi_to_seconds)
# Normalizing Data Across Seasons
cols_to_norm = skater_stats[4:]


def normalize(col):
    return (col - col.min()) / (col.max() - col.min())  # normalization formula


def normalize_cols(df):
    for col_name in cols_to_norm:
        df[f'{col_name}_norm'] = normalize(df[col_name])
    return df


df_final = normalize_cols(stats_df)

In [4]:
 # function to calculate distance between two points
def calc_distance(p1, p2):
    dist_vec = p1 - p2
    dist = np.sqrt(np.sum(dist_vec ** 2))
    return dist

# function to find the DataFrame row of a player given id and season
def find_player(player_id, season, df):
    for index, row in df.iterrows():
        if season == row['Season'] and player_id == row['ID']:
            return row
    
# function to find the normalized stats of a player given id and season
def current_player_stats(current_player_id, current_player_season, df):
    return np.array([
    (df.loc[(df['ID'] == current_player_id) & (df['Season'] == current_player_season), 'G_norm']).item(),
    (df.loc[(df['ID'] == current_player_id) & (df['Season'] == current_player_season), 'A_norm']).item(),
    (df.loc[(df['ID'] == current_player_id) & (df['Season'] == current_player_season), '+/-_norm']).item(),
    (df.loc[(df['ID'] == current_player_id) & (df['Season'] == current_player_season), 'PIM_norm']).item(),
    (df.loc[(df['ID'] == current_player_id) & (df['Season'] == current_player_season), 'PPP_norm']).item(),
    (df.loc[(df['ID'] == current_player_id) & (df['Season'] == current_player_season), 'ATOI_norm']).item(),
    (df.loc[(df['ID'] == current_player_id) & (df['Season'] == current_player_season), 'S_norm']).item(),
    (df.loc[(df['ID'] == current_player_id) & (df['Season'] == current_player_season), 'BLK_norm']).item(),
    (df.loc[(df['ID'] == current_player_id) & (df['Season'] == current_player_season), 'HIT_norm']).item()
    ])

In [5]:
season_list = [f'{str(x)}-{str(x + 1)[2:]}' for x in range(2007, 2024)]

def player_comparison_tool(current_player_id, current_player_season, df):
    
    if ((df['Season'] == current_player_season) & (df['ID'] == current_player_id)).any() == False:
        print(f"Can't find player with id {current_player_id} and season {current_player_season}")
        return None

    for row in df.itertuples():
        if current_player_season == row.Season and current_player_id == row.ID:
            break
            
    current_player_vector = current_player_stats(current_player_id, current_player_season, df)
    print(f"Projecting player_id {current_player_id} for season {season_list[season_list.index(current_player_season) + 1]}")

    player_distance = []

    weighted_numbers = [1, 1, 1, 1, 1, 1, 1, 1, 1]

    for index, row in df.iterrows():
        compared_player_vector = np.array([
            row['G_norm'],
            row['A_norm'],
            row['+/-_norm'],
            row['PIM_norm'],
            row['PPP_norm'],
            row['ATOI_norm'],
            row['S_norm'],
            row['BLK_norm'],
            row['HIT_norm']
        ])

        vfunc = np.vectorize(calc_distance)
        distance_vec = vfunc(current_player_vector, compared_player_vector)
        weighted_distance = distance_vec * weighted_numbers
        number = np.sum(np.abs(weighted_distance)) / len(
            distance_vec)  # standardized singular number that compares players 
        player_distance.append(number)

    df['distance'] = player_distance
    ranked_df = df.sort_values('distance')

    scoring_stats = skater_stats[4:]

    projected_stats = {}
    for col in scoring_stats:
        sum_stat = 0
        sum_weight = 0
        for index, row in ranked_df.iloc[1:11].iterrows():
            # skip over row if it is the 2023-24 season because we can't take the next season
            if row['Season'] == '2023-24':
                continue
            weight = 1 / row.distance
            next_season = season_list[season_list.index(current_player_season) + 1]
            # return row of player next season stats 
            player_next_season = find_player(row['ID'], next_season, df)
            # If player next season is not found (retire or otherwise) then skip
            if player_next_season is None:
                continue
            # Use getattr() method to access a variable attribute of an object (the row of the player next season stats)
            sum_stat += player_next_season[col] * weight
            sum_weight += weight
        projected_stats['ID'] = current_player_id
        projected_stats['proj_season'] = season_list[season_list.index(current_player_season) + 1]
        projected_stats[f'proj_{col}'] = sum_stat / sum_weight

    return projected_stats


In [13]:
# Projecting 2024-25 Season Stats

ten_players = [51580, 12823, 15976, 92452, 77112, 25577, 52255, 49538, 32753, 97120]

filtered_df = df_final[df_final['Season'] == '2021-22']
player_ids_2024 = filtered_df['ID'].tolist()
player_ids_2024

final_projections = []

for player_id in player_ids_2024:
    current_player_id = player_id
    current_player_season = '2021-22'
    projections = player_comparison_tool(current_player_id, current_player_season, df_final)
    if projections is None:
        continue
    final_projections.append(projections)


Projecting player_id 10771 for season 2022-23
Projecting player_id 51282 for season 2022-23
Projecting player_id 71582 for season 2022-23


KeyboardInterrupt: 

In [16]:
cols_to_round = ['proj_G', 'proj_A', 'proj_+/-', 'proj_PIM', 'proj_PPP', 'proj_ATOI', 'proj_S', 'proj_BLK', 'proj_HIT']
# convert final projections into a dataframe to split
proj_df = pd.DataFrame(data=final_projections)

# merge dataframes on player_id column and season_ids
df_final_unique = df_final.drop_duplicates(subset='ID', keep='first')
final_df = pd.merge(proj_df, df_final_unique[['ID', 'Player']],  on='ID')

# put player name column in the front
player_col = final_df.pop('Player')
final_df.insert(1, 'Player', player_col)

# round projected values
final_df[cols_to_round] = final_df[cols_to_round].round(0).astype(int)
final_df

,ID,Player,proj_season,proj_G,proj_A,proj_+/-,proj_PIM,proj_PPP,proj_ATOI,proj_S,proj_BLK,proj_HIT
0,10771,Noel Acciari,2022-23,8,8,-12,19,1,805,90,35,84
1,51282,Calen Addison,2022-23,1,1,0,3,0,874,12,9,7


[49696,
 96176,
 97365,
 20591,
 95023,
 88158,
 70073,
 51607,
 89354,
 54761,
 44343,
 73306,
 73291,
 90794,
 97419,
 14018,
 80322,
 73766,
 86421,
 95525,
 94902,
 52870,
 47157,
 61352,
 55168,
 50494,
 50272,
 21478,
 80634,
 96585,
 39862,
 34579,
 83424,
 78204,
 74402,
 40103,
 10217,
 86231,
 78096,
 50906,
 71138,
 33704,
 15210,
 34944,
 82350,
 18075,
 62267,
 77302,
 68435,
 33222,
 80286,
 71149,
 34475,
 35856,
 38751,
 64802,
 10875,
 25388,
 53998,
 96680,
 34714,
 41639,
 95726,
 36500,
 70690,
 82869,
 70155,
 73302,
 50054,
 91240,
 81400,
 38145,
 96236,
 26942,
 82584,
 23534,
 17309,
 38724,
 31368,
 85208,
 27883,
 68392,
 38570,
 69336,
 74723,
 29439,
 27863,
 47737,
 53403,
 50622,
 36154,
 45553,
 42976,
 91456,
 90111,
 37790,
 81824,
 96590,
 59678,
 20248,
 20194,
 47494,
 79265,
 26747,
 53151,
 83589,
 13461,
 89940,
 36013,
 55053,
 42751,
 80874,
 67209,
 84827,
 68971,
 64459,
 46547,
 48033,
 89470,
 76078,
 38432,
 13124,
 86371,
 23889,
 46393,


NameError: name 'ranked_df' is not defined